# Base analysis file for Ariane quantitative runs

In [1]:
#import required libraries
import pandas as pd
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt

In [2]:
xr.open_dataset("/ocean/rbeutel/MOAD/analysis-becca/Ariane/practice_run/15sep17/ariane_statistics_quantitative.nc")

<xarray.Dataset>
Dimensions:   (imt_reg: 310, jmt_reg: 286, kmt_reg: 40, nb_sect: 1)
Dimensions without coordinates: imt_reg, jmt_reg, kmt_reg, nb_sect
Data variables:
    xy_zonal  (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_mer    (nb_sect, jmt_reg, imt_reg) float32 ...
    xz_zonal  (nb_sect, kmt_reg, imt_reg) float32 ...
    xz_vert   (nb_sect, kmt_reg, imt_reg) float32 ...
    yz_mer    (nb_sect, kmt_reg, jmt_reg) float32 ...
    yz_vert   (nb_sect, kmt_reg, jmt_reg) float32 ...
    xy_uh     (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_vh     (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_zuh    (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_z2uh   (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_zvh    (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_z2vh   (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_tuh    (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_t2uh   (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_tvh    (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_t2vh   (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_suh    (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_s2uh   (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_svh    (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_s2vh   (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_ruh    (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_r2uh   (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_rvh    (nb_sect, jmt_reg, imt_reg) float32 ...
    xy_r2vh   (nb_sect, jmt_reg, imt_reg) float32 ...
    tmask     (kmt_reg, jmt_reg, imt_reg) float32 ...
Attributes:
    key_roms:                  .FALSE.
    key_mars:                  .FALSE.
    key_symphonie:             .FALSE.
    key_B2C_grid:              .FALSE.
    key_sequential:            .TRUE.
    key_alltracers:            .TRUE.
    key_ascii_outputs:         .TRUE.
    key_iU_jV_kW:              .FALSE.
    key_read_age:              .FALSE.
    mode:                      quantitative
    forback:                   backward
    bin:                       nobin
    init_final:                init
    nmax:                      100000
    tunit:                     3600.0
    ntfic:                     1
    tcyc:                      259200.0
    key_approximatesigma:      .FALSE.
    key_computesigma:          .TRUE.
    zsigma:                    100.0
    memory_log:                .TRUE.
    output_netcdf_large_file:  .FALSE.
    key_interp_temporal:       .FALSE.
    maxcycles:                 1
    key_2dquant:               .FALSE.
    key_eco:                   .TRUE.
    key_reducmem:              .TRUE.
    imt_reg_start:             1
    imt_reg_end:               310
    imt_reg:                   310
    jmt_reg_start:             182
    jmt_reg_end:               467
    jmt_reg:                   286
    kmt_reg_start:             1
    kmt_reg_end:               40
    kmt_reg:                   40
    key_unitm3:                .TRUE.
    key_nointerpolstats:       .FALSE.
    max_transport:             1000.0
    lmin:                      1
    lmax:                      24
    imt:                       398
    jmt:                       898
    kmt:                       40
    lmt:                       72
    key_computew:              .FALSE.
    w_surf_option:             
    key_partialsteps:          .TRUE.
    key_jfold:                 .FALSE.
    pivot:                     T
    key_periodic:              .FALSE.
    dir_mesh:                  /home/sallen/MEOPAR/grid/
    fn_mesh:                   mesh_mask201702.nc
    nc_var_xx_tt:              glamt
    nc_var_xx_uu:              glamu
    nc_var_yy_tt:              gphit
    nc_var_yy_vv:              gphiv
    nc_var_zz_ww:              gdepw_0
    nc_var_e2u:                e2u
    nc_var_e1v:                e1v
    nc_var_e1t:                e1t
    nc_var_e2t:                e2t
    nc_var_e3t:                e3t_0
    nc_var_tmask:              tmask
    nc_mask_val:               0.0
    c_dir_zo:                  Links
    c_pre